In [ ]:
graph_bin = "CoauthorPhysics.bin"

In [ ]:
!nvcc --version
import torch

In [ ]:
!pip install dgl-cu110
import dgl

In [ ]:
#loader
from dgl import save_graphs, load_graphs

def load_graph(name, split):
    k_path = r"/kaggle/input/dgl-inbuilt-graphs/"
    graph = load_graphs(k_path + name)[0][0]
    label = graph.ndata['label']
    feat = graph.ndata['feat']
    in_feat = graph.ndata['feat'].shape[1]
    out_feat = int(max(label)+1)
    mask = torch.BoolTensor(graph.num_nodes())
    mask[:] = False
    split_idx = int((graph.num_nodes()*split))
    mask[:split_idx] = True
    train_mask = mask
    test_mask = torch.logical_not(train_mask)
    return graph, feat, label, train_mask, test_mask, in_feat, out_feat

In [ ]:
g, features, labels, train_mask, test_mask, in_feat, out_feat = load_graph(graph_bin,0.8)

In [ ]:
import dgl
import torch
import torch.nn.functional as F
import time
import numpy as np
import psutil
dgl.seed(0)
torch.manual_seed(0)
dgl.random.seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

IN_FEAT = in_feat
H_FEAT = 128
OUT_FEAT = out_feat

BACKBONE = 'gcn' #for logging only

In [ ]:
# set config
if CONFIG == 0: #no ed
    EDROP_DA = False #dont care
    EDROPOUT = 0
elif CONFIG == 1: #random ed
    EDROP_DA = False
    EDROPOUT = 0.2
elif CONFIG == 2: #DAw ed
    EDROP_DA = True
    EDROPOUT = 0.2
else: #default config
    EDROP_DA = True
    EDROPOUT = 0.2
    
RUNS = 20
NDROPOUT = 0.5
EPOCHS = 500
NORM = False
N_BLOCKS = [1,2,3,4]
ROTATE_BY = 50
PRINT_EVERY = 10

In [ ]:
import torch.nn as nn
import random
from random import randrange

class RandomEdgeDropout(nn.Module):
    def __init__(self, da=False, dropout = 0.2, device='cuda:0'):
        super(RandomEdgeDropout, self).__init__()
        self.dropout = dropout
        self.da = da

    def forward(self, g):
        g = g.local_var()
        g = g.remove_self_loop()
        
        if not self.da:
            num_edges2drop = int(g.num_edges()*self.dropout)
            edges2drop = [randrange(g.num_edges()) for _ in range(num_edges2drop)]
        else:
            droppable = [idx for idx,i in enumerate(g.edata['droppable'].tolist()) if i==1]
            num_edges2drop = int(len(droppable)*self.dropout)
            edges2drop = random.sample(droppable, num_edges2drop)
        
        g.remove_edges(torch.tensor(edges2drop).to(device))
        g = dgl.add_self_loop(g)
        return g

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv, SAGEConv

class Residual(nn.Module):
    def __init__(self, in_feat, out_feat, aggr_mode="linear"):
        super(Residual, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.aggr_mode = aggr_mode
        self.conv = GraphConv(self.in_feat, self.out_feat, norm='both', weight=True, bias=True)

        if aggr_mode=="linear":
            self.aggr = nn.Linear(self.out_feat + self.in_feat, self.out_feat)
        if aggr_mode=="gcn":
            self.aggr = GraphConv(self.out_feat + self.in_feat, self.out_feat, norm='both', weight=True, bias=True)
        if aggr_mode=="sum":
            pass #do nothing here, handle in forward pass
        if aggr_mode=="max":
            pass #just let it pass bro

    def forward(self, graph, features):
        x = x_skip = features
        x = self.conv(graph, x)
        if self.aggr_mode=="linear":
            x = torch.cat([x,x_skip],dim=1)
            x = self.aggr(x)
        if self.aggr_mode=="gcn":
            x = torch.cat([x,x_skip],dim=1)
            x = self.aggr(graph,x)
        if self.aggr_mode=="sum":
            x += x_skip
        if self.aggr_mode=="max":
            x = torch.max(x,x_skip)
        return x


class JumpingKnowledge(nn.Module): 
    def __init__(self, in_feat, out_feat):
        super(JumpingKnowledge, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.conv = GraphConv(self.in_feat, self.out_feat, norm='both', weight=True, bias=True)

    def forward(self, graph, features):
        features_transformed = self.conv(graph, features)
        return features_transformed, features


class JumpingKnowledgeAggregator(nn.Module):
    def __init__(self, in_feat, h_feat, out_feat, num_layers, aggr_mode="linear"):
        super(JumpingKnowledgeAggregator, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.h_feat = h_feat
        self.num_layers = num_layers
        self.aggr_mode = aggr_mode

        if aggr_mode=="linear":
            self.aggr = nn.Linear(self.in_feat + self.h_feat*(self.num_layers-1) + self.out_feat, self.out_feat)

    def forward(self, feat_list):
        if self.aggr_mode=="linear":
            feat = torch.cat(feat_list,dim=1)
            feat = self.aggr(feat)
        if self.aggr_mode=="max":
            feat = torch.max(feat_list)
        if self.aggr_mode=="sum":
            feat = torch.sum(torch.stack(feat_list, dim=0),dim=0)
        return feat

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv, SAGEConv

class ResNet(nn.Module):
    def __init__(
        self, 
        graph, 
        in_feat, 
        h_feat, 
        out_feat, 
        n_blocks,
        edropout = 0.2,
        edropout_degAw = True, 
        ndropout = 0.2,
        norm = True, 
        device = 'cuda:0'
        ):
        super(ResNet, self).__init__()
        self.in_feat = in_feat
        self.h_feat = h_feat
        self.out_feat = out_feat
        
        self.edropout = edropout
        self.edropout_degAw = edropout_degAw
        self.ndropout = ndropout
        
        self.norm = norm
        self.n_blocks = n_blocks
        self.g = graph
        
        self.device = device
        self.layers = nn.ModuleList()
        
        self.ed = RandomEdgeDropout(da=True, dropout = self.edropout, device=self.device)
        self.lrelu = nn.LeakyReLU(negative_slope=0.1, inplace=False)

        
        if self.n_blocks==1:
            self.layers.append(Residual(self.in_feat, self.out_feat, aggr_mode="gcn"))
        else:
            for idx in range(self.n_blocks-1):
                if (idx==0):
                    self.layers.append(Residual(self.in_feat, self.h_feat, aggr_mode="gcn"))
                else:
                    self.layers.append(Residual(self.h_feat, self.h_feat, aggr_mode="gcn")) 
            self.layers.append(Residual(self.h_feat, self.out_feat, aggr_mode="gcn"))            
        self.fc = nn.Linear(self.out_feat, self.out_feat)
        self.bn = torch.nn.BatchNorm1d(self.out_feat)
    
    def forward(self, g, features):
        x = features
        for lyr in self.layers:
            with g.local_scope():
                if self.edropout != 0:
                    g = self.ed(g)
                x = self.lrelu(lyr(g, x))
                if self.ndropout != None:
                    x = F.dropout(x, self.ndropout, training=self.training)
        # x = self.fc(x)
        if self.norm:
            x = self.bn(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv, SAGEConv

class JKNet(nn.Module):
    def __init__(
        self, 
        graph, 
        in_feat, 
        h_feat, 
        out_feat, 
        n_blocks,
        edropout = 0.2,
        edropout_degAw = True, 
        ndropout = 0.2,
        norm = True, 
        device = 'cuda:0'
        ):
        super(JKNet, self).__init__()
        self.in_feat = in_feat
        self.h_feat = h_feat
        self.out_feat = out_feat
        
        self.edropout = edropout
        self.edropout_degAw = edropout_degAw
        self.ndropout = ndropout
        
        self.norm = norm
        self.n_blocks = n_blocks
        self.g = graph
        
        self.device = device
        self.layers = nn.ModuleList()
        self.aggr = JumpingKnowledgeAggregator(self.in_feat, self.h_feat, self.out_feat, self.n_blocks, aggr_mode="linear")
        
        self.ed = RandomEdgeDropout(da=True, dropout = self.edropout, device=self.device)
        self.lrelu = nn.LeakyReLU(negative_slope=0.1, inplace=False)

        
        if self.n_blocks==1:
            self.layers.append(JumpingKnowledge(self.in_feat, self.out_feat))      
        else:
            for idx in range(self.n_blocks-1):
                if (idx==0):
                    self.layers.append(JumpingKnowledge(self.in_feat, self.h_feat))
                else:
                    self.layers.append(JumpingKnowledge(self.h_feat, self.h_feat)) 
            self.layers.append(JumpingKnowledge(self.h_feat, self.out_feat)) 
                
        self.fc = nn.Linear(self.out_feat, self.out_feat)
        self.bn = torch.nn.BatchNorm1d(self.out_feat)    

    
    def forward(self, g, features):
        jumps = []
        x = features
        for lyr in self.layers:
            with g.local_scope():
                if self.edropout != 0:
                    g = self.ed(g)
                x,j = lyr(g, x)
                x = self.lrelu(x)
                jumps.append(j)
                if self.ndropout != None:
                    x = F.dropout(x, self.ndropout, training=self.training)
        jumps.append(x)
        x = self.aggr(jumps)
        # x = self.fc(x)
        if self.norm:
            x = self.bn(x)
        return x

In [ ]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
g = dgl.add_self_loop(g)
g = g.to(device)
features = features.to(device)
labels = labels.to(device)

In [ ]:
smry = {}
print(BACKBONE)
for N_BLOCK in N_BLOCKS:
    print('=' ,N_BLOCK,'layers ','='*42)
    best_scores =[]
    for run in range(RUNS):
        RUN_ID = run+1
        print('-' ,RUN_ID, '-'*50)

        net = JKNet(g, IN_FEAT, H_FEAT, OUT_FEAT, N_BLOCK, EDROPOUT, EDROP_DA, NDROPOUT, NORM, device).to(device)

        optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
        dur = []

        best_score = 0
        best_epoch = 0

        for epoch in range(EPOCHS):
            if epoch >=3:
                t0 = time.time()

            net.train()

            logits = net(g, features)

            logp = F.log_softmax(logits, 1)
            loss = F.nll_loss(logp[train_mask], labels[train_mask])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if epoch >=3:
                dur.append(time.time() - t0)

            acc = evaluate(net, g, features, labels, test_mask)
            if epoch%PRINT_EVERY==0:
                print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(epoch, loss.item(), acc, np.mean(dur)))

            if best_score < acc:
                best_score = acc
                best_epoch = epoch

        print("Best Test Acc {:.4f} at Epoch {:05d}".format(best_score, best_epoch))
        best_scores.append(best_score)

        del net
        train_mask = torch.tensor(train_mask.tolist()[ROTATE_BY:] + train_mask.tolist()[:ROTATE_BY]).bool().to(device)
        test_mask = torch.tensor(test_mask.tolist()[ROTATE_BY:] + test_mask.tolist()[:ROTATE_BY]).bool().to(device)
        
    #summarize
    import statistics as stats

    best_scores.sort()
    if CONFIG==2:
        best_scores.reverse()
    best_scores = best_scores[:10]
    best_scores = [score for score in best_scores]
    

    mean = stats.mean(best_scores)
    stdev = stats.stdev(best_scores)
    
    print("Summary:")
    print(round(mean,4),"±",round(stdev,4))
    smry[N_BLOCK] = str(round(mean,4)) + "±" + str(round(stdev,4))

In [ ]:
smry